# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных
Данные геологоразведки трёх регионов находятся в файлах:\
id — уникальный идентификатор скважины;\
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);\
product — объём запасов в скважине (тыс. баррелей).

Условия задачи:\
Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).\
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.\
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.\
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.\
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

Импортируем необходимые библиотеки для нашего исследования и предоставленные файлы

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
from scipy import stats as st
from numpy.random import RandomState
pd.options.display.float_format = '{:,.2f}'.format
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats as st
pd.options.mode.chained_assignment = None

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv') 
data_2 = pd.read_csv('/datasets/geo_data_1.csv') 
data_3 = pd.read_csv('/datasets/geo_data_2.csv') 

Посмотрим общую информацию и распечатает по 10 строк из каждой таблицы.

In [3]:
data_1.head(10) # выведем первые 10 строк

,id,f0,f1,f2,product
0,txEyH,0.71,-0.50,1.22,105.28
1,2acmU,1.33,-0.34,4.37,73.04
2,409Wp,1.02,0.15,1.42,85.27
3,iJLyR,-0.03,0.14,2.98,168.62
4,Xdl7t,1.99,0.16,4.75,154.04
5,wX4Hy,0.97,0.49,-0.74,64.74
6,tL6pL,0.65,0.53,1.78,49.06
7,BYPU6,-0.40,0.81,-5.62,72.94
8,j9Oui,0.64,-0.55,2.37,113.36
9,OLuZU,2.17,0.56,9.44,127.91


In [4]:
data_1.info() # выводим информацию о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_1.duplicated().sum() # проверка на дубликаты

0

In [6]:
data_2.head(10) # выведем первые 10 строк

,id,f0,f1,f2,product
0,kBEdx,-15.00,-8.28,-0.01,3.18
1,62mP7,14.27,-3.48,1.00,26.95
2,vyE1P,6.26,-5.95,5.00,134.77
3,KcrkZ,-13.08,-11.51,5.00,137.95
4,AHL4O,12.70,-8.15,5.00,134.77
5,HHckp,-3.33,-2.21,3.00,84.04
6,h5Ujo,-11.14,-10.13,4.00,110.99
7,muH9x,4.23,-0.00,2.00,53.91
8,YiRkx,13.36,-0.33,5.00,134.77
9,jG6Gi,1.07,-11.03,5.00,137.95


In [7]:
data_2.info() # выводим информацию о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_2.duplicated().sum() # проверка на дубликаты

0

In [9]:
data_3.head(10) # выведем первые 10 строк

,id,f0,f1,f2,product
0,fwXo0,-1.15,0.96,-0.83,27.76
1,WJtFt,0.26,0.27,-2.53,56.07
2,ovLUW,0.19,0.29,-5.59,62.87
3,q6cA6,2.24,-0.55,0.93,114.57
4,WPMUX,-0.52,1.72,5.90,149.60
5,LzZXx,-0.76,0.71,2.59,90.22
6,WBHRv,-0.57,0.32,1.77,45.64
7,XO8fn,-1.91,-2.46,-0.18,72.48
8,ybmQ5,1.78,-0.28,3.00,106.62
9,OilcN,-1.21,-0.44,5.92,52.95


In [10]:
data_3.info() # выводим информацию о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
data_3.duplicated().sum() # проверка на дубликаты

0

Просмотрев данные видим, что пропуски отсутствуют, дубликаты отсутствуют, типы данных верные, будем двигаться дальше.

## Обучение и проверка модели

Сперва нам необходимо разбить данные на обучающую и валидационную выборки. В задании нам приведен масштаб 75 на 25. Далее сделаем признаки взвешеными и после этого обучим модель линейной регрессии. Чтобы сделать удобно для трех таблиц сразу сделаем функцию, которая принимает на вход нужные данные и на выходе печатает средний запас предсказанного сырья, RMSE и реального сырья.

In [12]:
# создадим функцию
def region_linear_model(data):
    data_valid, data_train = train_test_split(data, test_size=0.75, random_state=12345)
    features_train = data_train.drop(['product', 'id'], axis=1)
    target_train = data_train['product']
    features_valid = data_valid.drop(['product', 'id'], axis=1)
    target_valid = data_valid['product']
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler() 
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric]) 
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    model = LinearRegression() 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    rmse = mean_squared_error(target_valid, predictions_valid)**0.5
    result = np.zeros(shape=(len(predictions_valid), 2))
    result[:, 0] = target_valid
    result[:, 1] = predictions_valid
    print('Средний запас сырья:', target_valid.mean())
    print('Предсказанный cредний запас сырья:', predictions_valid.mean())
    print('Корень среднеквадратичной ошибки RMSE:', rmse)
    print('Среднее абсолютное отклонение MAE:', mean_absolute_error(target_valid, predictions_valid))
    return result

In [13]:
region_predictions_1 = region_linear_model(data_1) # распечатаем результаты и сохраним отдельно таблицы для будущих расчетов
data_1_values = pd.DataFrame(region_predictions_1)
data_1_values.columns = ['target', 'predicted']

Средний запас сырья: 92.87310438882189
Предсказанный cредний запас сырья: 92.68042230714772
Корень среднеквадратичной ошибки RMSE: 37.68019646464799
Среднее абсолютное отклонение MAE: 30.998255825289256


In [14]:
region_predictions_2 = region_linear_model(data_2)
data_2_values = pd.DataFrame(region_predictions_2)
data_2_values.columns = ['target', 'predicted']

Средний запас сырья: 68.85836122606449
Предсказанный cредний запас сырья: 68.85982082622293
Корень среднеквадратичной ошибки RMSE: 0.8873287354658539
Среднее абсолютное отклонение MAE: 0.7158667523420444


In [15]:
region_predictions_3 = region_linear_model(data_3)
data_3_values = pd.DataFrame(region_predictions_3)
data_3_values.columns = ['target', 'predicted']

Средний запас сырья: 95.01017700303149
Предсказанный cредний запас сырья: 95.06093851120133
Корень среднеквадратичной ошибки RMSE: 40.11167877627781
Среднее абсолютное отклонение MAE: 32.905615384278285


Анализируя модели можно увидеть, что у первой и третьей показатель средней ошибки большой 37 и 32 соотв, зато у второй модели показатель самый низкий - 0,88. Показатель средней абсолютной так же большой в первой и третьей, а во второй близок к средней квадратичной. Запасы сырья во всех случаях предсказаны примерно одинаково на валидацонных выборках.

## Подготовка к расчёту прибыли

Рассчитаем необходимый минимальный объём запасов сырья для безубыточной добыми и сравним со средними запасами во всех регионах.

In [16]:
data_1['product'].mean() # средний запас в регионе 1

92.50000000000001

In [17]:
data_2['product'].mean() # средний запас в регионе 2

68.82500000000002

In [18]:
data_3['product'].mean() # средний запас в регионе 3

95.00000000000004

In [19]:
# рассчет среднего безубыточного объема запасов
drill_points = 200
n_drills = 500
budget = 10000000000
one_product_price = 450000
product_needed = budget / one_product_price / drill_points
print('Минимальный объём сырья для безубыточной разработки:', math.ceil(product_needed))

Минимальный объём сырья для безубыточной разработки: 112


В регионах среднее запасов меньше, чем бюджет на безубыточную разработку, поэтому для поиска скважит нужно выбрать лучшие. Для безубыточной добычи из 200 точек нам необходим объем в среднем 112 на каждую.

## Расчёт прибыли и рисков 

Сначала создадим функцию для рассчета прибыли, затем используем Bootstrap для анализа прибыли и рисков.

In [20]:
# функция расчета выручки
def profit(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    income_top = one_product_price * selected.sum()
    
    return income_top - budget

In [37]:
# функция метода bootstrap
def bootstrap(target, predictions):

    state = np.random.RandomState(12345)
    bootstrap_samples = 1000 
    
    values = []
    target = target.reset_index(drop=True)
    count = 0 
    
    for i in range(bootstrap_samples):
        target_subsample = target.sample(n=500, replace=True, random_state=state) 
        probs_subsample = pd.Series(predictions)[target_subsample.index]
        profit_sum = profit(target_subsample, probs_subsample, 200)
     
        
        if profit_sum < 0:
            count += 1
        values.append(profit_sum)
                
    values = pd.Series(values)
    lower = values.quantile(0.025) 
    upper = values.quantile(0.975) 
    profit_mean = values.mean()
    risk = (count / bootstrap_samples) * 100 
    return lower, upper, profit_mean, risk

In [38]:
# используем функцию к каждому району
lower_1, upper_1, profit_mean_1, risk_1 = bootstrap(data_1_values['target'], data_1_values['predicted'])
lower_2, upper_2, profit_mean_2, risk_2 = bootstrap(data_2_values['target'], data_2_values['predicted'])
lower_3, upper_3, profit_mean_3, risk_3 = bootstrap(data_3_values['target'], data_3_values['predicted'])

In [40]:
# печатаем результаты
tabel = {'region_1':[profit_mean_1, lower_1, upper_1, risk_1],
        'region_2':[profit_mean_2, lower_2, upper_2, risk_2],
        'region_3':[profit_mean_3, lower_3, upper_3, risk_3]}
table_df = pd.DataFrame(tabel)
table_df.index = (['Средняя прибыль', 'Доверительный интервал (начало)',
                    'Доверительный интервал (конец)', 'Риск убытков'])
table_df

,region_1,region_2,region_3
Средняя прибыль,"468,757,230.74","491,757,264.39","355,594,873.37"
Доверительный интервал (начало),"-85,513,325.78","81,624,191.17","-223,696,673.37"
Доверительный интервал (конец),"1,025,766,579.27","925,837,646.57","942,181,246.76"
Риск убытков,4.60,0.90,9.50


Из таблицы видим, что по рискам первый и третий регион проигрывают второму. Нас интересует риск не более 2,5%, с такими параметрами нам подходит только второй, у которого риск ра, плюс у него выручка выше, чем у остальных, поэтому выбор здесь однозначный.